## Transfer Learning of Xception for Multiple Watch Features at Once

At this point script does not run properly. A custom class is required to reset the custom generators, after the end of the first epoch.

In [51]:
import itertools
import os

import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications

from keras import backend as K

In [52]:
# dimensions of our images.
img_width, img_height = 299, 299

# dimensions of our images.
img_width, img_height = 299, 299
n_features = 8

top_model_weights_path = 'bottleneck_xception_model.h5'
train_data_dir = 'multi_class_testing/train/'
validation_data_dir = 'multi_class_testing/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

batch_size = 8

__Set up Multiclass Testing__

In [53]:
watch_df = pd.read_csv('all_watch_info_with_indicators.csv')
watch_dict = dict()

Replace These Features with something better

In [54]:
for image_name in watch_df['new_image_name']:
    watch_dict[image_name+'.jpg'] =  watch_df[
        watch_df['new_image_name'] == image_name][
            ['is_gold','is_silver','is_strap',
             'is_rotating','is_chronograph',
             'is_square','is_round','is_male']].values[0]
    

Multiclass Helper Functions

In [55]:
def multiclasses_getter(x, i, gen):
    count = 0
    y = np.zeros((batch_size, n_features), dtype=np.int8)
    
    # Feature not working
    if np.shape(x)[0] < gen.batch_size:
        steps = np.shape(x)[0]
    else:
        steps = gen.batch_size
        
    idx = (i) * gen.batch_size
    
    for f in gen.filenames[idx : idx + steps]:
        k = f.split('/')[-1]
        y[count,:] = watch_dict[k] 
        count += 1
    return y

In [56]:
def multiclass_flow_train(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i,train_generator)
    
def multiclass_flow_test(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i, test_generator)
      

Create Multi-lable metrics (Recovered from Keras 1.x)

In [7]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

Create Generators

In [19]:
datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary')

Found 7368 images belonging to 2 classes.
Found 3152 images belonging to 2 classes.


Load Standard Xception Model

In [9]:
model = applications.xception.Xception(include_top=False, weights='imagenet')

Build "Custom" Model

In [10]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(n_features, activation="sigmoid"))
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 16392     
Total params: 20,877,872
Trainable params: 20,823,344
Non-trainable params: 54,528
_________________________________________________________________


Freeze Xception Layers

In [11]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

Compile

In [12]:
watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy",precision,recall])

Fit 

In [13]:
epochs = 10

In [14]:
# fine-tune the model
watch_model.fit_generator(
        multiclass_flow_train(train_generator, multiclasses_getter),
        steps_per_epoch=nb_train_samples / batch_size,
        epochs=epochs,
        validation_data = multiclass_flow_test(test_generator, multiclasses_getter),
        validation_steps=nb_validation_samples / batch_size)

watch_model.save('xception_multiclass1.h5')

Epoch 1/10


Exception in thread Thread-26:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/utils/data_utils.py", line 568, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-6-963edb77df7c>", line 3, in multiclass_flow_train
    yield x, multiclasses_getter(x, i,train_generator)
  File "<ipython-input-5-b689636c25c4>", line 15, in multiclasses_getter
    y[count,:] = watch_dict[k]
KeyError: '2874_BreitlingGalactic32a71356L2q5792cd.jpg'



StopIteration: 

In [ ]:
watch_model = load_model('xception_multiclass1.h5', 
                         custom_objects={'precision': precision,'recall':recall})

In [ ]:
#Unfreeze a few more layers and allow to run

epochs = 50

watch_model.layers[0].summary()

# Freeze convolutional layers
for layer in watch_model.layers[0].layers[-3:]:
    layer.trainable = True


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy",precision,recall])


# fine-tune the model
watch_model.fit_generator(
        multiclass_flow_train(train_generator, multiclasses_getter),
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = multiclass_flow_test(test_generator, multiclasses_getter),
        validation_steps=nb_validation_samples // batch_size)

In [ ]:
#watch_model.sample_weights('xception_multiclass_unfrozen_convos1_weights.h5')
watch_model.save('xception_multiclass_unfrozen_convos1.h5')

In [ ]:
del watch_model

In [ ]:
watch_model = load_model('xception_multiclass_unfrozen_convos1.h5', 
                         custom_objects={'precision': precision,'recall':recall})